# Going to start woith some basic stats

In [188]:
import pandas as pd
import numpy as np
import pyodbc
import warnings
from slugify import slugify

import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import random

random.seed(4407)
# from pycaret.classification import setup, compare_models, predict_model, finalize_model, save_model

# Suppress warnings
warnings.simplefilter("ignore")

In [189]:
server = 'localhost\SQLEXPRESS'
database = 'nba_game_data'

In [190]:
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';')
cursor = cnxn.cursor()

sql = """
WITH base AS (
Select *, [AST] + [REB] + [PTS] as PTS_REB_AST
FROM 
    [nba_game_data].[dbo].[PlayerGameLogs]
),base2 as (
SELECT 
    [SEASON_YEAR],
    [PLAYER_ID],
    [PLAYER_NAME],
    [TEAM_ABBREVIATION],
    oppAbrv,
    yearSeason,
    [GAME_ID],
    [GAME_DATE],
    [PTS],
    PTS_REB_AST,
    LAG([PTS], 1) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE) AS Last_Game_Pts,
    LAG([PTS], 1) OVER (PARTITION BY PLAYER_ID, pgl.oppAbrv ORDER BY GAME_DATE) AS Last_Game_Pts_Against_Opp,
    LAG(PTS_REB_AST, 1) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE) AS Last_Game_Pts_Reb_Ast,
    LAG(PTS_REB_AST, 1) OVER (PARTITION BY PLAYER_ID, pgl.oppAbrv ORDER BY GAME_DATE) AS Last_Game_Pts_Reb_Ast_Against_Opp,
    STDEV(PTS) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS Pts_Volatility,
    AVG(PTS) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN 10 PRECEDING AND 6 PRECEDING) -
    AVG(PTS) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING) AS Pts_Momentum,
    AVG(PTS) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING) AS Pts_MovingAverage5Games,
    STDEV(PTS_REB_AST) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) AS Pts_Reb_Ast_Volatility,
    AVG(PTS_REB_AST) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN 10 PRECEDING AND 6 PRECEDING) -
    AVG(PTS_REB_AST) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN 5 PRECEDING AND 1 PRECEDING) AS Pts_Reb_Ast_Momentum,
    AVG(PTS_REB_AST) OVER (PARTITION BY PLAYER_ID, SEASON_YEAR ORDER BY GAME_DATE ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING) AS Pts_Reb_Ast_MovingAverage5Games,

    AVG(PTS) OVER (PARTITION BY PLAYER_ID, pgl.oppAbrv ORDER BY GAME_DATE ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING) AS Pts_Moving_Avg_Season_Against_Opp,
    AVG(PTS_REB_AST) OVER (PARTITION BY PLAYER_ID, pgl.oppAbrv ORDER BY GAME_DATE ROWS BETWEEN 4 PRECEDING AND 1 PRECEDING) AS Pts_Reb_Ast_Moving_Avg_Season_Against_Opp
FROM 
    base pgl

--WHERE 
--    yearSeason >2023
    
)
SELECT 
PLAYER_NAME
,PLAYER_ID
,oppAbrv
,SEASON_YEAR
,GAME_DATE
,PTS
,PTS_REB_AST
,Last_Game_Pts
,Last_Game_Pts_Against_Opp
,Last_Game_Pts_Reb_Ast
,Last_Game_Pts_Reb_Ast_Against_Opp
,Pts_Volatility
,Pts_Momentum
,Pts_MovingAverage5Games
,Pts_Reb_Ast_Volatility
,Pts_Reb_Ast_Momentum
,Pts_Reb_Ast_MovingAverage5Games
,Pts_Moving_Avg_Season_Against_Opp
,Pts_Reb_Ast_Moving_Avg_Season_Against_Opp



FROM base2
WHERE 
    yearSeason >= 2024
ORDER BY 
    GAME_DATE;
  """
df = pd.read_sql(sql,cnxn)
df['Player_Slug'] = df['PLAYER_NAME'].apply(slugify)
df = df.drop(columns=['PLAYER_NAME'])

In [191]:
# df.loc[df['PLAYER_NAME'].str.contains('Chet')].sort_values(by='Date').iloc[-60:]

In [192]:
def compute_rsi(series, window_length=7):
    # Calculate the difference but shift the result down to ensure the current row's change is not included
    delta = series.diff().shift()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate the EWMA of gains and losses, now effectively excluding the current row
    avg_gain = gain.ewm(com=window_length - 1, min_periods=window_length).mean()
    avg_loss = loss.ewm(com=window_length - 1, min_periods=window_length).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

# Apply the RSI calculation to each player and season group using transform to ensure DataFrame alignment
df['Pts_RSI'] = df.groupby(['PLAYER_ID', 'SEASON_YEAR'])['PTS'].transform(lambda x: compute_rsi(x))
df['Pts_Reb_Ast_RSI'] = df.groupby(['PLAYER_ID', 'SEASON_YEAR'])['PTS_REB_AST'].transform(lambda x: compute_rsi(x))

In [193]:
df = df.drop(columns=['SEASON_YEAR','PLAYER_ID','oppAbrv'])

In [194]:
df_roto = pd.read_csv(r'C:\Users\bobby\Downloads\all_rotowire_stats.csv')

df_roto['line_Type'] = ''
for i, row in df_roto.iterrows():
    if row['Market_Name'] == 'Rebounds':
        df_roto.loc[i,'line_Type'] ='REB'
    elif row['Market_Name'] == '3PT Made':
        df_roto.loc[i,'line_Type'] ='FG3M'
    elif row['Market_Name'] == 'Assists':
        df_roto.loc[i,'line_Type'] ='AST'
    elif row['Market_Name'] == 'Blocks':
        df_roto.loc[i,'line_Type'] ='BLK'
    elif row['Market_Name'] == 'PTS+REB+AST':
        df_roto.loc[i,'line_Type'] ='PTS+REB+AST'
    elif row['Market_Name'] == 'Turnovers':
        df_roto.loc[i,'line_Type'] ='TOV'
    elif row['Market_Name'] == 'Points':
        df_roto.loc[i,'line_Type'] ='PTS'
    elif row['Market_Name'] == 'Steals':
        df_roto.loc[i,'line_Type'] ='STL'
        
df_roto['Opponent'] = df_roto['Opponent'].apply(lambda x: x.replace('@','').strip())
df_roto['Player_Slug'] = df_roto['Player'].apply(slugify)

In [195]:
df_lines = df_roto.loc[df_roto['line_Type'].isin(['PTS+REB+AST','PTS'])][['Player_Slug','Date','line_Type','Line','Site_Less','Site_More']]

In [196]:
df['Date'] = pd.to_datetime(df['GAME_DATE'])
df_lines['Date'] = pd.to_datetime(df_lines['Date'])

In [197]:
df['Player_Slug'] = df['Player_Slug'].replace('-jr','',regex=True)

In [198]:
data = pd.merge(df_lines, df, on = ['Player_Slug','Date'],how='inner')

In [199]:
data['market_prs'] = (data['line_Type']=='PTS+REB+AST').astype(int)
data['market_pts'] = (data['line_Type']=='PTS').astype(int)

In [200]:
# Define the function to determine the target
def determine_target(row):
    if row['line_Type'] == 'PTS':
        return row['PTS'] > row['Line']
    elif row['line_Type'] == 'PTS+REB+AST':
        return row['PTS_REB_AST'] > row['Line']
    return False

def determine_market(row):
    if row['line_Type'] == 'PTS':
        return row['PTS'] > row['Line']
    elif row['line_Type'] == 'PTS+REB+AST':
        return row['PTS_REB_AST'] > row['Line']
    return False

# Apply the function to create the 'target' column
data['target'] = data.apply(determine_target, axis=1)
data['target']=data['target'].astype(int)

In [201]:
data = data.drop(columns=['Player_Slug','Date','line_Type','GAME_DATE','PTS','PTS_REB_AST'])

In [202]:
data.fillna(-999, inplace=True)

In [203]:
data[data.isna().any(axis=1)]#.iloc[:,:20]#['PLAYER_ID']

,Line,Site_Less,Site_More,Last_Game_Pts,Last_Game_Pts_Against_Opp,Last_Game_Pts_Reb_Ast,Last_Game_Pts_Reb_Ast_Against_Opp,Pts_Volatility,Pts_Momentum,Pts_MovingAverage5Games,Pts_Reb_Ast_Volatility,Pts_Reb_Ast_Momentum,Pts_Reb_Ast_MovingAverage5Games,Pts_Moving_Avg_Season_Against_Opp,Pts_Reb_Ast_Moving_Avg_Season_Against_Opp,Pts_RSI,Pts_Reb_Ast_RSI,market_prs,market_pts,target


# model selection


In [204]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

In [205]:
# Define features and target
X = data.drop(columns=['target'])
y = data['target']

# Split the data into training and holdout sets
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.1, random_state=42)

# Define the models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'KNN': KNeighborsClassifier(),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    'AdaBoost': AdaBoostClassifier(),
    'CatBoost': CatBoostClassifier(verbose=0),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'SVM': SVC(probability=True),
    'Naive Bayes': GaussianNB(),
    'Voting Classifier': VotingClassifier(estimators=[
        ('lr', LogisticRegression(max_iter=1000)),
        ('dt', DecisionTreeClassifier()),
        ('knn', KNeighborsClassifier()),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='logloss')),
        ('adb', AdaBoostClassifier()),
        ('cb', CatBoostClassifier(verbose=0))
    ], voting='soft')
}

# Define metrics
metrics = {
    'Model': [],
    'Recall': [],
    'Precision': [],
    'F1': [],
    'Accuracy': [],
    'AUC': []
}

# Perform 10-fold cross-validation and evaluate models
kf = KFold(n_splits=20, shuffle=True, random_state=42)
for name, model in models.items():
    recall_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='recall')
    precision_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='precision')
    f1_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='f1')
    accuracy_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='accuracy')
    auc_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring='roc_auc')

    metrics['Model'].append(name)
    metrics['Recall'].append(recall_scores.mean())
    metrics['Precision'].append(precision_scores.mean())
    metrics['F1'].append(f1_scores.mean())
    metrics['Accuracy'].append(accuracy_scores.mean())
    metrics['AUC'].append(auc_scores.mean())

# Create a dataframe to display the metrics
metrics_df = pd.DataFrame(metrics)

# Evaluate models on the holdout set
holdout_metrics = {
    'Model': [],
    'Recall': [],
    'Precision': [],
    'F1': [],
    'Accuracy': [],
    'AUC': []
}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_holdout)
    y_prob = model.predict_proba(X_holdout)[:, 1]

    holdout_metrics['Model'].append(name)
    holdout_metrics['Recall'].append(recall_score(y_holdout, y_pred))
    holdout_metrics['Precision'].append(precision_score(y_holdout, y_pred))
    holdout_metrics['F1'].append(f1_score(y_holdout, y_pred))
    holdout_metrics['Accuracy'].append(accuracy_score(y_holdout, y_pred))
    holdout_metrics['AUC'].append(roc_auc_score(y_holdout, y_prob))

# Create a dataframe to display holdout metrics
holdout_metrics_df = pd.DataFrame(holdout_metrics)




KeyboardInterrupt: 

In [ ]:
len(X)

In [ ]:
len(X_holdout)

In [ ]:
metrics_df

In [ ]:
holdout_metrics_df

In [ ]:
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score, confusion_matrix


In [ ]:
# Function to get the prediction matrix for a given model name
def get_prediction_matrix(model_name):
    if model_name not in models:
        raise ValueError(f"Model {model_name} is not defined.")
    
    model = models[model_name]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_holdout)
    y_prob = model.predict_proba(X_holdout)[:, 1]
    
    recall = recall_score(y_holdout, y_pred)
    precision = precision_score(y_holdout, y_pred)
    f1 = f1_score(y_holdout, y_pred)
    accuracy = accuracy_score(y_holdout, y_pred)
    auc = roc_auc_score(y_holdout, y_prob)
    
    tn, fp, fn, tp = confusion_matrix(y_holdout, y_pred).ravel()
    
    prediction_matrix = {
        'Model': model_name,
        'Recall': recall,
        'Precision': precision,
        'F1': f1,
        'Accuracy': accuracy,
        'AUC': auc,
        'True Positives': tp,
        'True Negatives': tn,
        'False Positives': fp,
        'False Negatives': fn
    }
    
    return pd.DataFrame([prediction_matrix])

# Example usage
model_name = 'Random Forest'
prediction_matrix_df = get_prediction_matrix(model_name)
prediction_matrix_df
# import ace_tools as tools; tools.display_dataframe_to_user(name="Prediction Matrix", dataframe=prediction_matrix_df)

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)
dd = pd.DataFrame(model.predict_proba(X_holdout))
dd['actual'] = y


In [ ]:
dd = pd.DataFrame(model.predict_proba(X_holdout))

In [ ]:
dd['actual'] = y
dd.sort_values(by=0)

In [213]:
import numpy as np
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from skopt.space import Real, Integer
from sklearn.metrics import f1_score, make_scorer

# Define the model
model = RandomForestClassifier(random_state=42)

# Define the search space
param_space = {
    'n_estimators': Integer(10, 500),
    'max_depth': Integer(1, 50),
    'min_samples_split': Integer(2, 10),
    'min_samples_leaf': Integer(1, 10),
    'max_features': Real(0.1, 1.0)
}

# Define the scoring function
f1_scorer = make_scorer(f1_score, average='weighted')

# Set up the BayesSearchCV
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces=param_space,
    n_iter=100,  # Number of iterations
    cv=5,  # 5-fold cross-validation
    random_state=42,
    n_jobs=-1,
    scoring=f1_scorer
)

# Fit the model
bayes_search.fit(X_train, y_train)

# Print the best parameters
print("Best hyperparameters:", bayes_search.best_params_)
print("Best score (F1):", bayes_search.best_score_)

# Evaluate the model on the test set
best_model = bayes_search.best_estimator_
test_score = f1_score(y_holdout, best_model.predict(X_holdout), average='weighted')
print("Test set F1 score:", test_score)

Best hyperparameters: OrderedDict([('max_depth', 22), ('max_features', 1.0), ('min_samples_leaf', 1), ('min_samples_split', 5), ('n_estimators', 229)])
Best score (F1): 0.7624657712292873
Test set F1 score: 0.7671548840964135


In [214]:
best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=22, max_features=1.0, min_samples_split=5,
                       n_estimators=229, random_state=42)

In [215]:
best_model.classes_

array([0, 1])

In [224]:
pd.DataFrame([best_model.feature_importances_
,best_model.feature_names_in_]).T

,0,1
0,0.050633,Line
1,0.0406,Site_Less
2,0.036976,Site_More
3,0.041692,Last_Game_Pts
4,0.047308,Last_Game_Pts_Against_Opp
5,0.04921,Last_Game_Pts_Reb_Ast
6,0.053166,Last_Game_Pts_Reb_Ast_Against_Opp
7,0.088136,Pts_Volatility
8,0.039836,Pts_Momentum
9,0.049073,Pts_MovingAverage5Games


In [216]:
# best_model.predict(X_holdout)
dd = pd.DataFrame(best_model.predict_proba(X_holdout),columns=best_model.classes_)
dd['actual'] = y
dd
dd.sort_values(by=0)

,0,1,actual
0,0.084799,0.915201,0
49,0.138683,0.861317,0
4,0.160636,0.839364,1
30,0.188646,0.811354,1
45,0.209680,0.790320,1
36,0.217977,0.782023,0
43,0.235683,0.764317,0
28,0.239759,0.760241,0
3,0.243627,0.756373,0
11,0.252412,0.747588,1
